Task01 赛题理解

### 赛题理解
此次比赛是新闻推荐场景下的用户行为预测挑战赛，要求根据用户浏览点击新闻文章的数据信息预测用户未来的点击行为，即用户的最后一次点击的新闻文章。

### 数据概况
该数据来自某新闻APP平台的用户交互数据，包括30万用户（user），近300万次点击，共36万篇不同的新闻文章，同时每篇新闻文章有对应的embedding向量表示。从中抽取20万用户的点击日志数据作为训练集，5万用户的点击日志数据作为测试集A，5万用户的点击日志数据作为测试集B，具体数据表和参数如下：

**数据表**：

`train_click_log.csv`：训练集用户点击日志

`testA_click_log.csv`：测试集用户点击日志

`articles.csv`：新闻文章信息数据表

`articles_emb.csv`：新闻文章embedding向量表示

`sample_submit.csv`：提交样例文件

**字段表**：

|    Field  |  Description    |
| ---- | ---- |
|   user_id |用户id |
|click_article_id |点击文章id |
|click_timestamp | 点击时间戳 |
|click_environment |点击环境  |
|click_deviceGroup  |点击设备组|
|click_os |点击操作系统 |
|click_country |点击城市 |
|click_region |点击地区 |
|click_referrer_type | 点击来源类型 |
|article_id |文章id，与click_article_id相对应 |
|category_id | 文章类型id |
|created_at_ts | 文章创建时间戳 |
|words_count | 文章字数 |
|emb_1,emb_2,…,emb_249 |文章embedding向量表示   |   

### 评价方式理解
查看`sample.submit.csv`可知我们需要提交的格式是对测试集中的每个用户，按照预测的点击概率从大到小，给出五篇文章的推荐结果，如：

| user_id |article_1|article_2|article_3|article_4|article_5|
| ---- | ---- | ---- | ---- | ---- | ---- |
| 200000 | 1 | 2 | 3 | 4 | 5 |

每个用户实际最后一次点击的文章只有一篇，我们就看预测的5篇文章中会不会命中真实答案。评价指标为`MRR（Mean Reciprocal Rank）`：首先对测试集中每个用户计算用户得分：

$$score(user)=\sum\limits_{k=1}^5\frac{s(user,k)}{k}$$

其中，如果对该user的预测结果中的第k个结果命中该user的最后一次购买数据，则$s(user,k)=1$，否则为0；这个数再除以k应该就是对排序结果的惩罚，即命中的结果排序越靠后，惩罚越大。提交的最后得分就是所有用户的$score(user)$的平均值。

### 赛题理解

此次新闻推荐比赛和普通的结构化比赛不太一样（如此前组队学习的贷款违约预测）：一是在目标上，要预测出用户最后一次可能点击的文章的Top5，而不是一个回归数值或分类概率；二是比赛数据不是简单的特征+标签的形式，而是基于真实业务场景拿到的用户的点击日志；可利用的数据文件也有多个，如新闻文章信息表，文章embedding向量表示，这就需要多个数据表的merge；由于要根据用户的历史数据预测其最后一次点击，因此在划分训练集和验证集时要避免时间穿越问题。

因此我们的思考的方向就是，把该预测问题转成一个监督学习的问题（特征+标签），然后进行ML，DL等建模预测。由于要按可能点击的概率大小给出用户最后一次点击文章的Top5，就可以转化成一个多分类问题，每篇文章是一类，我们要预测用户点击落入某一类的概率。进一步地，若直接预测对36万篇文章的点击概率，即36个分类，会显得比较困难。那可以先从海量文章中用一些简单的规则或者算法初筛出用户可能感兴趣的文章，再从初筛的结果中进行精排序，得到Top5，这既是目前推荐系统领域最常用的**召回+排序**策略。

### ItemCF Baseline实现

In [1]:
# import packages
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
warnings.filterwarnings('ignore')

In [2]:
data_path = './data_raw/'
save_path = './tmp_results/'

In [3]:
# df 节约内存的一个标配函数
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

In [4]:
#读取采样或者全量数据
# debug模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    """
        训练集中采样一部分数据调试
        data_path: 原数据的存储路径
        sample_nums: 采样数目（这里由于机器的内存限制，可以采样用户做）
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))  #去除重复项
    return all_click

In [5]:
# 读取点击数据，这里分成线上和线下，如果是为了获取线上提交结果应该讲测试集中的点击数据合并到总的数据中
# 如果是为了线下验证模型的有效性或者特征的有效性，可以只使用训练集
def get_all_click_df(data_path='./data_raw/', offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = trn_click.append(tst_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [6]:
# 全量训练集
all_click_df = get_all_click_df(offline=False)
all_click_df = reduce_mem(all_click_df)

-- Mem. usage decreased to 46.65 Mb (62.5% reduction),time spend:0.01 min


In [7]:
# 获取用户-文章-点击时间字典
# 根据点击时间先后获取用户的点击文章序列   {user1: {item1: time1, item2: time2..}...}
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

In [8]:
# 获取近期点击最多的文章（所有文章中）
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

In [9]:
# itemCF的物品相似度计算
def itemcf_sim(df):
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0)
                
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_

In [10]:
i2i_sim = itemcf_sim(all_click_df)

100%|██████████| 250000/250000 [00:35<00:00, 7009.63it/s] 


In [11]:
# itemCF的文章推荐
# 基于商品的召回 i2i

def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: [item1: time1, item2: time2..]...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全        
        return: 召回的文章列表 [(item1:score1), (item2: score2)...}]
        注意: 基于物品的协同过滤，在多路召回部分会加上关联规则的召回策略
    """
    
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {user_id for user_id, _ in user_hist_items}
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue
                
            item_rank.setdefault(j, 0)
            item_rank[j] +=  wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

In [12]:
# 给每个用户根据物品的协同过滤推荐文章
import collections
# 定义
user_recall_items_dict = collections.defaultdict(dict)

# 获取 用户 - 文章 - 点击时间的字典
user_item_time_dict = get_user_item_time(all_click_df)

# 去取文章相似度
i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

# 相似文章的数量
sim_item_topk = 10

# 召回文章数量
recall_item_num = 10

# 用户热度补全
item_topk_click = get_item_topk_click(all_click_df, k=50)

for user in tqdm(all_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, 
                                                        sim_item_topk, recall_item_num, item_topk_click)

100%|██████████| 250000/250000 [35:06<00:00, 118.68it/s]


In [13]:
# 召回字典转换成df
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])

100%|██████████| 250000/250000 [00:05<00:00, 45606.53it/s]


In [14]:
# 生成提交文件

def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [15]:
# 获取测试集
tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
tst_users = tst_click['user_id'].unique()

# 从所有的召回数据中将测试集中的用户选出来
tst_recall = recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall, topk=5, model_name='itemcf_baseline')

ItemCF baseline 线上成绩0.1026

**Task01小结**：

* 了解赛题的基本情况，包括赛题，数据，评估指标
* 跑通ItemCF baseline并在线上提交